# Supply and Demand: Machine learning Testing

### Objective 
Establish a correlation between the CUST_ID in the supply data and the potential amount of SKU_ID against the POS_Name based on BLDG_ID and SqFt_est using a random forest machine learning model.

### Conceptual Diagram
SUPPLY
Table A -> Table B -> Table C[CUST_ID]

DEMAND
Table X -> Table Y -> Table Z[POS_NAME]

### Outline
1. Data Preparation
2. Feature Engineering 
3. Splitting Data
4. Random Forest Model
5. Model Evaluation

---
# Data Preparation

In [1]:
import pandas as pd

In [48]:
# SUPPLY TABLES FILEPATHS
table_A_path = '../data/0_key.csv'
table_B_path = '../data/0_sku.csv'
table_C_path = '../data/1_dist.csv'

# DEMAND TABLES FILEPATHS
table_X_path = '../data/04-SECT.csv'
table_Y_path = '../data/03-POS-A.csv'
table_Z_path = '../data/03-POS-B.csv'

# CORRELATION DATA FILEPATHS
table_5_path = '../data/05-MATRIX.csv'

In [17]:
def read_and_clean_csv(file_path):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)
    
    # Drop rows with NaN values
    df_cleaned = df.dropna()
    
    return df_cleaned

In [33]:
# Read and clean the CSV files
table_A_df = read_and_clean_csv(table_A_path)
table_B_df = read_and_clean_csv(table_B_path)
table_C_df = read_and_clean_csv(table_C_path)
table_X_df = read_and_clean_csv(table_X_path)
table_Y_df = read_and_clean_csv(table_Y_path)
table_Z_df = read_and_clean_csv(table_Z_path)

##### Merging into two major tabless

In [34]:
merged_supply = pd.merge(pd.merge(table_A_df, table_B_df, on='schema_id'), table_C_df, on=['schema_id'])


In [35]:
merged_supply.head(1)

,schema_id,name_desc,avg_stock,avg_price_x,avg_mktshare,SKU_ID,sku_name,avg_price_y,supply_desc_x,distribution,production,CUST_ID,CUST__NAME,supply_desc_y,CUST_ADDRESS
0,S01,Soybeans,802075,0.22,176456,1001020,soy nuts,4.49,food packaging,commercial,unfermented,2001005,Anu Resources,food packaging,"11757 Katy Freeway, Suite 1300, Houston 77079 ..."


In [37]:
merged_supply.describe()

,avg_stock,avg_price_x,avg_mktshare,SKU_ID,avg_price_y,CUST_ID
count,227.000000,227.000000,227.000000,2.270000e+02,227.000000,2.270000e+02
mean,239575.722467,3.079824,72552.383260,1.001013e+06,12.115242,2.001022e+06
std,326686.508202,2.997312,48023.638918,7.956962e+00,20.023863,1.185245e+01
min,3185.000000,0.190000,37213.000000,1.001000e+06,0.670000,2.001000e+06
25%,5390.000000,0.190000,40519.000000,1.001005e+06,4.490000,2.001012e+06
50%,15925.000000,2.540000,40519.000000,1.001014e+06,6.320000,2.001023e+06
75%,683667.000000,6.900000,129897.000000,1.001019e+06,7.370000,2.001029e+06
max,802075.000000,13.660000,176456.000000,1.001026e+06,90.000000,2.001043e+06


In [38]:
merged_demand = pd.merge(table_Y_df, table_Z_df, on='BLDG_ID')


In [39]:
merged_demand.head(1)

,BLDG_ID,SqFtCd_x,SqFt_est,POS_Landuse,sectorid,POS_StateClass,POS_StateClassCd,center_x,center_y,POS_Contact,...,POS_NAICS,POS_North American Industry Classification,POS_TYPE_focus,POS_Revenue,Owner_Ethnicity,Owner_Type,Owner_Established,Owner_CreditScore,SqFtCd_y,SqFt_range
0,528,UNDER 5000SF,2317.5609,COMMERCIAL,2,COMMERCIAL,F1,-95.3519,29.76013,IRMA G GALVAN,...,72251117,FULL-SERVICE RESTAURANTS,RESTAURANTS,"$500,000-$1 MILLION",HISPANIC,FIRM,1990,B+,3.0,"2,500 - 4,999"


In [40]:
merged_demand.describe()

,BLDG_ID,SqFt_est,sectorid,center_x,center_y,Latitude,Longitude,POS_SELSIC,POS_NAICS,Owner_Established,SqFtCd_y
count,10.00000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,1.000000e+01,10.000000,10.000000
mean,732.10000,3472.381349,2.700000,-95.365770,29.755002,29.754990,-95.365780,543965.200000,6.814437e+07,2005.100000,2.900000
std,409.08528,1055.447572,4.321779,0.010086,0.004885,0.004878,0.010098,117870.113203,1.297601e+07,9.314863,1.523884
min,177.00000,2317.560900,0.000000,-95.377500,29.749080,29.749100,-95.377500,208501.000000,3.121400e+07,1990.000000,1.000000
25%,535.75000,2541.670882,0.000000,-95.376325,29.752147,29.752150,-95.376325,581208.000000,7.224100e+07,2002.000000,2.000000
50%,645.50000,3357.409565,0.000000,-95.364000,29.752885,29.752850,-95.364000,581208.000000,7.225112e+07,2005.000000,3.000000
75%,973.00000,4255.496708,3.500000,-95.357075,29.759498,29.759475,-95.357075,581277.750000,7.225112e+07,2012.000000,3.750000
max,1342.00000,4948.907364,11.000000,-95.351900,29.761980,29.762000,-95.351900,581301.000000,7.225112e+07,2016.000000,6.000000


---

# Feature Engineering

In [53]:
supply_categorical_columns = ['supply_desc_x','supply_desc_y','distribution','production']
demand_categorical_columns = ['POS_StateClass','POS_North American Industry Classification','POS_Revenue']

In [44]:
encoded_supply_df = pd.get_dummies(merged_supply, columns=supply_categorical_columns)

In [46]:
encoded_supply_df.head(3)

,schema_id,name_desc,avg_stock,avg_price_x,avg_mktshare,SKU_ID,sku_name,avg_price_y,CUST_ID,CUST__NAME,CUST_ADDRESS,supply_desc_x_food manufacturing,supply_desc_x_food packaging,supply_desc_y_food manufacturing,supply_desc_y_food packaging,distribution_commercial,distribution_wholesale,production_fermented,production_processed,production_unfermented
0,S01,Soybeans,802075,0.22,176456,1001020,soy nuts,4.49,2001005,Anu Resources,"11757 Katy Freeway, Suite 1300, Houston 77079 ...",0,1,0,1,1,0,0,0,1
1,S01,Soybeans,802075,0.22,176456,1001020,soy nuts,4.49,2001006,Al Hakeem Co,9318 Lynchester Dr Houston 77083 Texas United ...,0,1,0,1,1,0,0,0,1
2,S01,Soybeans,802075,0.22,176456,1001020,soy nuts,4.49,2001007,Lien Hoa B. Inc,7611 Summer Glen Ln Houston 77072 United States,0,1,0,1,1,0,0,0,1


In [54]:
encoded_demand_df = pd.get_dummies(merged_demand, columns=demand_categorical_columns)

In [55]:
encoded_demand_df.head(3)

,BLDG_ID,SqFtCd_x,SqFt_est,POS_Landuse,sectorid,POS_StateClassCd,center_x,center_y,POS_Contact,POS_Name,...,POS_StateClass_COMMERCIAL,POS_StateClass_COMMUNITY,POS_North American Industry Classification_DISTILLERIES,POS_North American Industry Classification_DRINKING PLACES ALCOHOLIC BEVERAGES,POS_North American Industry Classification_FULL-SERVICE RESTAURANTS,POS_Revenue_$1-2.5 MILLION,POS_Revenue_$2.5-5 MILLION,POS_Revenue_$5-10 MILLION,"POS_Revenue_$500,000-$1 MILLION","POS_Revenue_LESS THAN $500,000"
0,528,UNDER 5000SF,2317.560900,COMMERCIAL,2,F1,-95.3519,29.76013,IRMA G GALVAN,IRMA'S RESTAURANT,...,1,0,0,0,1,0,0,0,1,0
1,707,UNDER 5000SF,4948.907364,COMMERCIAL,4,F1,-95.3632,29.74908,LISA JUE,CHINA GARDEN,...,1,0,0,0,1,0,0,0,1,0
2,177,UNDER 5000SF,4861.121116,COMMERCIAL,10,F1,-95.3648,29.76156,ALLAN LEVINE,CULTIVATED F+B,...,1,0,0,0,1,0,1,0,0,0


---
# Splitting the Datasets

In [49]:
correlation_df = read_and_clean_csv(table_5_path)

In [50]:
correlation_df

,POS_TYPE_focus,S01,S02,S03,B01,B02,B03,Total
0,BAR,0.10,0.20,0.40,0.05,0.65,0.10,1.50
1,BEVERAGE STORES,0.00,0.00,0.10,0.00,0.00,0.00,0.10
2,BREWERY,0.00,0.00,0.10,0.00,0.00,0.05,0.15
3,CATERERS,0.25,0.25,0.25,0.25,0.25,0.25,1.50
4,CHAIN RESTAURANTS,0.00,0.10,0.40,0.65,0.00,0.00,1.15
5,ENTERTAINMENT,0.00,0.00,0.40,0.25,0.00,0.00,0.65
6,FOOD STORES,0.10,0.10,0.15,0.10,0.10,0.10,0.65
7,GENERAL STORES,0.10,0.10,0.10,0.10,0.00,0.10,0.50
8,HOTELS,0.10,0.85,0.10,0.25,0.10,0.10,1.50
9,PHARMACY STORES,0.00,0.00,0.05,0.00,0.00,0.05,0.10



Correlation values as features. This indicates the likelyhood of a buisness to have the a SKU_id under the schema_ID in stock. 

In [51]:
correlation_long = correlation_df.melt(id_vars=['POS_TYPE_focus'], var_name='schema_id', value_name='correlation')


In [57]:
merged_demand = pd.merge(encoded_demand_df, correlation_long, on=['POS_TYPE_focus'], how='left')


In [66]:
merged_demand['volume_sku'] = (
    (merged_demand['SqFt_est'] * merged_demand['correlation'])
    / (
        merged_demand['POS_Revenue_$1-2.5 MILLION'] * 100
        + merged_demand['POS_Revenue_$2.5-5 MILLION'] * 150
        + merged_demand['POS_Revenue_$5-10 MILLION'] * 200
        + merged_demand['POS_Revenue_$500,000-$1 MILLION'] * 50
        + merged_demand['POS_Revenue_LESS THAN $500,000'] * 25
    )
)

Creating the Feature Matrix 

In [80]:
merged_demand.shape

(70, 35)

In [81]:
encoded_supply_df.shape

(227, 20)

----


In [73]:
supply_features = encoded_supply_df[['schema_id','avg_stock','avg_price_x','avg_price_y', 'avg_mktshare']]

In [74]:
merged_data = pd.merge(merged_demand, supply_features, on='schema_id')

In [64]:
demand_features = merged_demand[['SqFt_est','correlation','sectorid']]

In [76]:
merged_data = pd.merge(merged_data, correlation_df, left_on='POS_TYPE_focus', right_on='POS_TYPE_focus', how='left')

In [77]:
X = pd.concat([supply_features, demand_features, merged_demand.drop(['POS_TYPE_focus'], axis=1)], axis=1)

In [78]:
target_variable = merged_demand['volume_sku']

In [71]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, target_variable, test_size=0.2, random_state=42)